In [1]:
from openai import OpenAI
import json, os, sys
import os
import re
import random

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def generate_questions(chunk, client, model="gpt-4o"):
    prompt = f"""
You are an expert in news analysis and skilled at creating multiple-choice questions that help to differentiate between articles. 
I have some summaries of news articles and I need to generate unique, comprehensive multiple-choice questions based on these summaries.

Your task is to generate questions that:
- Focusing on Kenneth Burke’s Dramatistic Pentad, which is a tool for analyzing stories and narratives.
- Help to differentiate between the articles.
- Are common to all articles, not specific to any one article.
- Are unique and not repeated in any other batch.
- Include "None of the above" as one of the options for each question.

Below is the format and example I need:

1. What was done?
   - A. The Chinese coast guard seized one of four food packs dropped by a Philippine military plane for Filipino navy personnel at a territorial outpost. After discovering the package contained food, they dumped it into the sea.
   - B. Philippine soldiers were reported to have pointed guns at Chinese coast guard personnel during a resupply mission to the grounded Sierra Madre ship. The Chinese coast guard responded to the resupply operation, which included food drops, by observing armed Philippine soldiers on the ship’s deck.
   - C. Turkish Foreign Minister Hakan Fidan began a trip to China and expressed priorities to support Hamas against Israel and increase trade with China, without condemning the Uyghur genocide.
   - D. The Philippines is collaborating with the United States and Japan to ensure the West Philippine Sea (WPS) remains free and safe amid tensions with Chinese maritime forces.
   - E. None of the above


Please generate as many unique and comprehensive questions as possible based on the given article summaries below. 
Make sure each question is designed to highlight differences between the articles.

Summaries: {chunk}

Generate as many as you can unique questions based on the above summaries.
"""
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [4]:
def process_batches(input_file, output_file, client, model):
    # Read the article summaries from the input JSON file
    with open(input_file, "r") as f:
        articles = json.load(f)

    # Shuffle the data
    # random.shuffle(articles)

    # Extract summaries from the articles
    article_summaries = [article["summary"] for article in articles]
    article_summaries = (
        article_summaries[200:300] + article_summaries[500:]
    )  # Limit to 200 summaries
    num_summaries = len(article_summaries)
    batch_size = 200

    responses = []

    # Process the article summaries in batches of 100
    for i in range(0, num_summaries, batch_size):
        batch = article_summaries[i : i + batch_size]
        chunk = json.dumps(batch)  # Convert the batch to a JSON string

        questions = generate_questions(chunk, client, model=model)
        responses.append(
            {
                "batch_start": i,
                "batch_end": min(i + batch_size - 1, num_summaries - 1),
                "questions": questions,
            }
        )

    # Write the generated questions to the output JSON file
    with open(output_file, "w") as f:
        json.dump(responses, f, indent=4)

In [5]:
input_file = "../data/QnA_data/combined_summary.json"
output_file = "output_questions.json"

# Assuming you have your OpenAI API client initialized as `client`
process_batches(input_file, output_file, client, model="gpt-4o")

In [6]:
def extract_questions(batch_questions):
    questions = {}
    q_count = 1
    for batch in batch_questions:
        batch_qs = batch["questions"].split("\n\n")
        for q in batch_qs:
            match = re.match(
                r"\d+\. (.+?)\n\s+- A\. (.+?)\n\s+- B\. (.+?)\n\s+- C\. (.+?)\n\s+- D\. (.+?)\n\s+- E\. (.+?)$",
                q.strip(),
                re.DOTALL,
            )
            if match:
                question = match.group(1).strip()
                choices = {
                    "A": match.group(2).strip(),
                    "B": match.group(3).strip(),
                    "C": match.group(4).strip(),
                    "D": match.group(5).strip(),
                    "E": match.group(6).strip(),
                }
                if question not in questions:
                    questions[question] = choices
    return questions


def convert_to_json2_format(unique_questions):
    formatted_questions = {}
    q_num = 1
    for question, choices in unique_questions.items():
        formatted_questions[f"Q{q_num}"] = {"question": question, "choices": choices}
        q_num += 1
    return formatted_questions


def process_json(input_json):
    unique_questions = extract_questions(input_json)
    return convert_to_json2_format(unique_questions)

In [7]:
# Load JSON 1
with open("output_questions.json", "r") as f:
    json1 = json.load(f)

# Process JSON 1 to get JSON 2 format
json2_format = process_json(json1)

# Save the result to a new JSON file
with open("formated_output_questions.json", "w") as f:
    json.dump(json2_format, f, indent=4)